### Q1 - Define function description

In [1]:
import random

known_weather_data = {
    'berlin': 20.0
}

def get_weather(city: str) -> str:
    city = city.strip().lower()

    if city in known_weather_data:
        return str(known_weather_data[city])

    return str(round(random.uniform(-5, 35), 1))

Remember that the message content sent to the LLM must be text, so the **function must return a string result**.

In [2]:
get_weather_tool = {
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get the current temperature (degrees celsius) in a given city",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "The city name"
                }
            },
            "required": ["city"],
        }
    }
}

Each of the properties contains the name of a parameter to use when calling the function with arguments, in this case `city`.

In [8]:
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()

client = OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=os.environ.get("GROQ_API_KEY")
)

In [9]:
import chat_assistant

tools = chat_assistant.Tools()
tools.add_tool(get_weather, get_weather_tool)

tools.get_tools()

[{'type': 'function',
  'function': {'name': 'get_weather',
   'description': 'Get the current temperature (degrees celsius) in a given city',
   'parameters': {'type': 'object',
    'properties': {'city': {'type': 'string', 'description': 'The city name'}},
    'required': ['city']}}}]

In [10]:
system_prompt = """
You are a weather assistant.
If the location requested is not a city, then tell the user you cannot provide a reliable answer.
Only when asked about a city use the get_weather function.
Otherwise, do not make a call to get_weather.
Once you have the required temperature data about a city, give a final answer.
""".strip()

chat_interface = chat_assistant.ChatInterface()

chat = chat_assistant.ChatAssistant(
    tools=tools,
    developer_prompt=system_prompt,
    chat_interface=chat_interface,
    client=client
)

In [11]:
chat.run()

Chat ended.


### Q2 - Adding another tool



In [12]:
def set_weather(city: str, temp: float) -> str:
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

In [20]:
set_weather_tool = {
    "type": "function",
    "function": {
        "name": "set_weather",
        "description": "Link a city to its current temperature",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "The city name"
                },
                "temp": {
                    "type": "number",
                    "description": "The temperature"
                }
            },
            "required": ["city", "temp"],
        }
    }
}

In [21]:
tools.add_tool(set_weather, set_weather_tool)
tools.get_tools()

[{'type': 'function',
  'function': {'name': 'get_weather',
   'description': 'Get the current temperature (degrees celsius) in a given city',
   'parameters': {'type': 'object',
    'properties': {'city': {'type': 'string', 'description': 'The city name'}},
    'required': ['city']}}},
 {'type': 'function',
  'function': {'name': 'set_weather',
   'description': 'Link a city to its current temperature',
   'parameters': {'type': 'object',
    'properties': {'city': {'type': 'string', 'description': 'The city name'},
     'temp': {'type': 'number', 'description': 'The temperature'}},
    'required': ['city', 'temp']}}}]

In [24]:
chat.run()

Chat ended.


In [25]:
known_weather_data

{'berlin': 20.0, 'madrid': 24}

### Q3 - FastMCP Installation

In [26]:
%pip install fastmcp

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached jsonschema-4.24.0-py3-none-any.whl.metadata (7.8 kB)
  Using cached jsonschema_specifications-2025.4.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached referencing-0.36.2-py3-none-any.whl.metadata (2.8 kB)
  Using cached cffi-1.17.1-cp313-cp313-win_amd64.whl.metadata (1.6 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
Using cached attrs-25.3.0-py3-none-any.whl (63 kB)
Using cached jsonschema-4.24.0-py3-none-any.whl (88 kB)
   ---------------------------------------- 0.0/3.4 MB ? eta -:--:--
   ---------------------------------------- 3.4/3.4 MB 53.4 MB/s eta 0:


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [27]:
# Linux: pip freeze | grep "fastmcp"
%pip freeze | findstr "fastmcp"

fastmcp==2.10.5
Note: you may need to restart the kernel to use updated packages.


### Q4 - Simple MCP Server

MCP stands for **Model-Context Protocol**. It allows LLMs communicate with different tools (like Qdrant). It's function calling, but one step further:

- A tool can export a list of functions it has.
- When we include the tool to our agent, we just need the link to the MCP server.

Simple weather MCP server implementation: [weather_server.py](./weather_server.py)

**Starting message**:  Starting MCP server 'Demo 🚀' with transport '**stdio**'

### Q5 - Protocol

There are different ways to communicate with an MCP server. Ours is currently running using standart input/output.
The protocol used is jsonrpc.

- Inicialization request.
```json
{"jsonrpc": "2.0", "id": 1, "method": "initialize", "params": {"protocolVersion": "2024-11-05", "capabilities": {"roots": {"listChanged": true}, "sampling": {}}, "clientInfo": {"name": "test-client", "version": "1.0.0"}}}
```

- Confirm initialization.
```json
{"jsonrpc": "2.0", "method": "notifications/initialized"}
```

- Ask for list of methods/tools.
```json
{"jsonrpc": "2.0", "id": 2, "method": "tools/list"}
```

Finally, asking the temperature in Berlin:
```json
{"jsonrpc": "2.0", "id": 3, "method": "tools/call", "params": {"name": "get_weather", "arguments": {"city": "Berlin"}}}
```

Response:
```json
{"jsonrpc":"2.0","id":3,"result":{"content":[{"type":"text","text":"20.0"}],"structuredContent":{"result":20.0},"isError":false}}
```

> The id is incremented to refer to new requests.


### Q6 - Client

In [33]:
from fastmcp import Client
import weather_server

async with Client(weather_server.mcp) as mcp_client:
    tools = await mcp_client.list_tools()

tools

[Tool(name='get_weather', title=None, description='Retrieves the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to retrieve weather data.\n\nReturns:\n    float: The temperature associated with the city.', inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'number'}}, 'required': ['result'], 'title': '_WrappedResult', 'type': 'object', 'x-fastmcp-wrap-result': True}, annotations=None, meta=None),
 Tool(name='set_weather', title=None, description="Sets the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to set the weather data.\n    temp (float): The temperature to associate with the city.\n\nReturns:\n    str: A confirmation string 'OK' indicating successful update.", inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}, 'temp': {'title': 'T